# DataNeeded

## 1 預處理

In [1]:
import pandas as pd
import json
import os
import re

In [2]:
# 讀取數據文件
Adata = pd.read_csv("[2] 2019320fin.csv", encoding="utf-8-sig")

Adata.head()

,id,city,ctripName,lon,lat,value,stars,price,isquality,strategymedal,...,差评,有图片,商务出差,朋友出游,情侣出游,家庭亲子,独自出行,代人预订,其他,房型类数
0,345025,北京,北京中奥马哥孛罗大酒店,116.408622,40.005020,4.8,5,926,1,0,...,192,217,3512.0,345.0,229.0,509.0,332.0,212.0,309.0,6.0
1,345029,北京,北京西单美爵酒店,116.374998,39.903426,4.5,4,788,0,0,...,174,275,2405.0,262.0,303.0,1068.0,169.0,136.0,244.0,14.0
2,346488,北京,北京北方佳苑饭店,116.412954,39.910324,4.3,4,528,1,0,...,304,203,1746.0,302.0,222.0,1050.0,203.0,161.0,416.0,10.0
3,347308,北京,北京王府井希尔顿酒店,116.413340,39.914840,4.6,0,1321,0,0,...,157,458,1536.0,403.0,381.0,1578.0,216.0,98.0,239.0,15.0
4,347310,北京,北京长城饭店(原喜来登长城饭店),116.463587,39.944516,4.4,5,758,1,0,...,605,404,4893.0,459.0,352.0,548.0,353.0,269.0,369.0,15.0



| column   name  | describe                                 | \|   | column name        | describe                                           |
| -------------- | ---------------------------------------- | ---- | ------------------ | -------------------------------------------------- |
| id             | 酒店編号(base ctrip)                     | \|   | input_QunarName    | 輸入搜索的酒店名                                   |
| city           | 所在熱門城市                             | \|   | get_qunarName      | 在Qunar平臺檢索後的酒店名                          |
| ctripArea      | 酒店所在區域(ctrip)                      | \|   | qunarValue         | 酒店評分(Qunar)                                    |
| ctripName      | 酒店名稱(ctrip)                          | \|   | qunarCommentsCount | 酒店評論數(Qunar)                                  |
| value          | 酒店總評分(ctrip)                        | \|   | qunarURL           | 酒店網址(Qunar)                                    |
| stars          | 國家評定等級(ctrip)                      | \|   | fliggy_inputName   | 輸入搜索的酒店名                                   |
| price          | 酒店價格(ctrip 2018-11-24)               | \|   | fliggyName         | 在Fliggy平臺檢索後的酒店名                         |
| isquality      | 是否品質保證(ctrip)                      | \|   | diffValue          | fliggy_inputName &   fliggy_inputName 的文本值差異 |
| strategymedal  | 是否特牌(ctrip)                          | \|   | isTrust            | 是否信用入住(fliggy)                               |
| goldmedal      | 是否金牌(ctrip)                          | \|   | fliggyPrice        | 酒店價格(fliggy)                                   |
| silvermedal    | 是否銀牌(ctrip)                          | \|   | fliggyValue        | 酒店評分(fliggy)                                   |
| diamondRank    | 攜程評定鑽石等級(ctrip)                  | \|   | fliggyvolume       | 酒店評論數量(fliggy)                               |
| judgementscore | 用戶推薦入住百分比(ctrip 2018-11-24)     | \|   | fliggyURL          | 網址(fliggy)                                       |
| commentscount  | 酒店評論數(ctrip 2018-11-24)             | \|   |                    |                                                    |
| labelscount    | 區域或酒店類型標籤數量(ctrip 2018-11-24) | \|   |                    |                                                    |
| strlabels      | 區域或酒店類型標籤內容(ctrip 2018-11-24) | \|   |                    |                                                    |
| servicescount  | 酒店服務項目數量(ctrip)                  | \|   |                    |                                                    |
| strservices    | 酒店服務項目內容                         | \|   |                    |                                                    |
| ctripURL       | 酒店網址(ctrip)                          | \|   |                    |                                                    |

---

## mathcing 2 

Exact Match X1 城市、X4星级，然后找价格在实验组价格的上下10%区间里的所有酒店，计算他们Diff_Rating=CtripRating-QunarRating, 然后选择Diff_Rating为中位数的哪一家作为match的对照组。

<font color="blue">- 攜程選優酒店 Lf ( 初步定義: 持 strategymedal的酒店)</font>

In [3]:
# 特徵選擇
selFeature = [
    "id",
    "city",
    "ctripName",
    "stars",
    "strategymedal",
    "goldmedal",
    "silvermedal",
    "price",
]


In [4]:
# 獲取優選酒店
# Lf = Adata[Adata.strategymedal == 1]
# 重新定义 優選酒店
Lf = Adata[(Adata.strategymedal + Adata.goldmedal + Adata.silvermedal) != 0]
# 优选酒店数量
rre = Lf.strategymedal.shape[0]
Lf = Lf[selFeature]
Lf.sort_values("id", inplace=True)

Lf.reset_index(inplace=True, drop=True)
print(Lf.shape)
Lf.head(3)

(2972, 8)


,id,city,ctripName,stars,strategymedal,goldmedal,silvermedal,price
0,344928,三亚,三亚亚龙湾万豪度假酒店,5,0,1,0,951
1,345001,上海,上海日航饭店,5,0,0,1,970
2,345008,广州,广州方圆奥克伍德豪景,0,0,1,0,1040


- <font color="blue"> 攜程非選優酒店 Rf

In [5]:
Rf = Adata[(Adata.strategymedal + Adata.goldmedal + Adata.silvermedal) == 0]
Rf = Rf[selFeature]
Rf.sort_values("id", inplace=True)

Rf.reset_index(inplace=True, drop=True)
print(Rf.shape)
Rf.head()

(12915, 8)


,id,city,ctripName,stars,strategymedal,goldmedal,silvermedal,price
0,141587,深圳,深雅酒店(深圳时尚店),0,0,0,0,121
1,344612,南京,南京卡特快捷宾馆(原摩卡庄园快捷酒店),0,0,0,0,158
2,344853,广州,广州富力君悦大酒店,5,0,0,0,1269
3,344857,广州,广州花园酒店,5,0,0,0,1166
4,344881,广州,广州中国大酒店,5,0,0,0,902


- <font color="blue">數據文件輸出

In [6]:
Lf.to_csv(r"H1\H1 Lf Ctrip and Qunar.csv",encoding="utf-8-sig")
Rf.to_csv(r"H1\H1 Rf Ctrip and Qunar.csv",encoding="utf-8-sig")

- match

In [11]:
#from math import radians, cos, sin, asin, sqrt
import pandas as pd
import json
import os
import re
import numpy as np


Lf = pd.read_csv(r"H1\H1 Lf Ctrip and Qunar.csv", encoding="utf-8-sig")
RRf = pd.read_csv(r"H1\H1 Rf Ctrip and Qunar.csv", encoding="utf-8-sig")


# ----------------------------------------------------hard match
num = 0
resultS = []
resultD = []
k = 1
for _ in Lf.id:
    Rf = RRf.copy()
    # match 所在城市
    Rf = Rf[Rf.city == Lf.loc[num].city]
    # match 酒店星级
    Rf = Rf[Rf.stars == Lf.loc[num].stars]
    t = Lf.loc[num].price

    # 价格差
    Rf = Rf[(abs(Rf.price-t))/t <= 0.1]
    resultS.append(Rf.shape[0])
    if Rf.shape[0] == 0:
        num += 1
        continue
    Rf["diff"] = abs(Rf.price-t)/t
#     print(Rf["diff"].min())
    Rf = Rf[Rf["diff"] == Rf["diff"].min()]

#     Rf["matching to id"] = Lf.loc[num].id
#     Rf["matching to name"] = Lf.loc[num].ctripName

    t = {
        "matchingNum": [k, k],
        "matchingLabel":[1,0],
        "id": [Lf.loc[num].id, Rf.id.tolist()[0]],
        "city": [Lf.loc[num].city, Rf.city.tolist()[0]],
        "hotelname": [Lf.loc[num].ctripName, Rf.ctripName.tolist()[0]],
#         "strategymedal": [Lf.loc[num].strategymedal, Rf.strategymedal.tolist()[0]],
#         "goldmedal": [Lf.loc[num].goldmedal, Rf.goldmedal.tolist()[0]],
#         "silvermedal": [Lf.loc[num].silvermedal, Rf.silvermedal.tolist()[0]],
        "stars": [Lf.loc[num].stars, Rf.stars.tolist()[0]],
        "hotelprice": [Lf.loc[num].price, Rf.price.tolist()[0]],
        "价格差百分比": [Rf["diff"].min()*100, Rf["diff"].min()*100]
        
    }

    num += 1
    k += 1
    resultD.append(pd.DataFrame(t))


resultD[0]

,matchingNum,matchingLabel,id,city,hotelname,stars,hotelprice,价格差百分比
0,1,1,345001,上海,上海日航饭店,5,970,0.206186
1,1,0,375528,上海,上海南新雅大酒店,5,968,0.206186


In [12]:
t1=resultS.count(0)
t2=len(resultS)
t3=max(resultS)
str1=f"匹配失败数量:{t2-t1},匹配失败数量:{t1},匹配最多数量:{t3}"

print(str1,"\n",resultS)

匹配失败数量:2897,匹配失败数量:75,匹配最多数量:353 
 [0, 10, 12, 20, 7, 14, 6, 145, 3, 6, 4, 3, 0, 0, 2, 0, 2, 19, 11, 1, 16, 0, 0, 8, 22, 4, 53, 0, 1, 3, 0, 99, 22, 25, 0, 0, 0, 13, 6, 8, 24, 8, 26, 10, 6, 13, 0, 16, 18, 6, 5, 11, 1, 2, 0, 8, 2, 2, 3, 3, 10, 1, 7, 8, 6, 53, 8, 1, 6, 1, 6, 4, 22, 0, 45, 6, 53, 51, 48, 0, 24, 2, 0, 8, 38, 2, 7, 13, 7, 8, 121, 6, 4, 0, 5, 4, 99, 163, 120, 1, 2, 9, 186, 10, 1, 2, 7, 44, 28, 161, 13, 98, 4, 0, 10, 0, 7, 15, 7, 32, 45, 1, 5, 140, 82, 7, 51, 27, 23, 6, 0, 85, 3, 2, 63, 5, 12, 5, 33, 2, 29, 79, 0, 12, 8, 126, 2, 4, 5, 14, 13, 5, 161, 260, 12, 12, 95, 3, 1, 122, 1, 5, 2, 6, 14, 0, 33, 71, 8, 22, 116, 30, 55, 135, 96, 21, 32, 71, 50, 10, 0, 4, 112, 140, 19, 328, 237, 277, 299, 220, 5, 92, 95, 95, 129, 46, 3, 153, 164, 39, 8, 9, 17, 2, 123, 51, 80, 11, 190, 7, 0, 150, 1, 276, 54, 17, 5, 5, 4, 0, 1, 16, 1, 69, 6, 4, 2, 9, 8, 67, 0, 114, 54, 163, 136, 98, 11, 2, 51, 0, 54, 17, 19, 39, 70, 147, 69, 36, 31, 50, 1, 70, 48, 146, 2, 58, 256, 2, 12, 104, 94, 49, 19, 30, 

In [13]:
# 將所有匹配到的酒店合併成一個DF 對象
fa = pd.concat(resultD, axis=0, sort=False)
fa=fa.reset_index(drop="index")
sete=[]
for i in range(fa.shape[0]):
    D=Adata[Adata.id==fa.iloc[i].id]
    sete.append(D)

In [14]:
fa.columns.tolist()

['matchingNum',
 'matchingLabel',
 'id',
 'city',
 'hotelname',
 'stars',
 'hotelprice',
 '价格差百分比']

In [15]:
matchinset=pd.concat(sete,axis=0)
matchinset.reset_index(drop="index",inplace=True)

In [17]:
outfile=pd.concat([matchinset,fa[["matchingNum","matchingLabel","价格差百分比"]]],axis=1)
outfile

,id,city,ctripName,lon,lat,value,stars,price,isquality,strategymedal,...,朋友出游,情侣出游,家庭亲子,独自出行,代人预订,其他,房型类数,matchingNum,matchingLabel,价格差百分比
0,345001,上海,上海日航饭店,121.437279,31.217559,4.4,5,970,0,0,...,687.0,519.0,720.0,383.0,469.0,441.0,8.0,1,1,0.206186
1,375528,上海,上海南新雅大酒店,121.477049,31.234396,4.5,5,968,0,0,...,566.0,330.0,1978.0,244.0,255.0,498.0,13.0,1,0,0.206186
2,345008,广州,广州方圆奥克伍德豪景,113.328640,23.129614,4.5,0,1040,0,0,...,60.0,83.0,175.0,33.0,32.0,35.0,7.0,2,1,2.980769
3,456162,广州,广州雅诗阁服务公寓,113.333313,23.135414,4.5,0,1071,0,0,...,36.0,24.0,76.0,12.0,20.0,23.0,9.0,2,0,2.980769
4,345012,广州,桔子水晶酒店(广州淘金店),113.290204,23.135711,4.8,0,826,1,1,...,481.0,816.0,657.0,276.0,108.0,235.0,16.0,3,1,0.242131
5,1317133,广州,广州大学城雅乐轩酒店,113.222249,23.397200,4.5,0,828,0,0,...,262.0,322.0,800.0,210.0,92.0,351.0,15.0,3,0,0.242131
6,345025,北京,北京中奥马哥孛罗大酒店,116.408622,40.005020,4.8,5,926,1,0,...,345.0,229.0,509.0,332.0,212.0,309.0,6.0,4,1,1.079914
7,374795,北京,北京伯豪瑞廷酒店,116.471174,39.914122,4.6,5,936,1,0,...,274.0,235.0,262.0,235.0,243.0,316.0,11.0,4,0,1.079914
8,345029,北京,北京西单美爵酒店,116.374998,39.903426,4.5,4,788,0,0,...,262.0,303.0,1068.0,169.0,136.0,244.0,14.0,5,1,0.126904
9,1194548,北京,北京锡华商务酒店,116.302536,39.989397,4.2,4,789,1,0,...,105.0,71.0,369.0,93.0,51.0,162.0,14.0,5,0,0.126904


In [18]:
outfile.to_csv("[3] matchHotel.csv",encoding="utf-8-sig")

In [19]:
filename="[3] matchHotel"

## 输出match 酒店

In [20]:
def tocsv2(filename):
    tager=pd.read_csv(f"{filename}.csv",encoding="utf-8-sig")

    x=tager.iloc[0::2,1:]
    x.reset_index(drop="index",inplace=True)

    y=tager.iloc[1::2,1:]
    y.columns=[(i+"_matched Result") for i in y.columns]
    y.reset_index(drop="index",inplace=True)

    z=pd.concat([x,y],axis=1)
    z.to_csv(f"{filename}_marge.csv",encoding="utf-8-sig")
    
tocsv2("[3] matchHotel")

---

## **在此 补充包含配对酒店的评论数据表文件夹 rating后,再进行下一步的操作

In [21]:
import os
filecsv = pd.read_csv("[3] matchHotel.csv", encoding="utf-8-sig")

def getDF(id):
    filename = f"{str(id)}.csv"
    with open("rating\\"+filename, "r", encoding="utf-8-sig") as f:
        ret = ""
        for i in range(200):
            ret += (f.readline())
        try:
            a = ret.split("commentAccount")
            if len(a) > 1:
                with open("rating2\\"+filename, "w", encoding="utf-8-sig") as fw:
                    fw.write("commentAccount" + a[1])
                return 0
            else:
                return id
        except:
            print(filename)


yy = list(set(filecsv.id.tolist()))
# print(len(yy))
re=[]
for i in yy:
    a=getDF(i)
    if a!=0:
        re.append(a)

FileNotFoundError: [Errno 2] No such file or directory: 'rating\\442372.csv'

## 处理并输出统计前100条评论标准差的数据

In [22]:
filecsv = pd.read_csv("[3] matchHotel.csv", encoding="utf-8-sig")

listname=["id","city","ctripName","matchingNum","matchingLabel","value","stars","price","价格差百分比"]
filecsv=filecsv[listname]
STD = []

# 将评论中前100条数据取出
for i in filecsv.id:
    try:
        t = pd.read_csv(f"{os.getcwd()}\\rating2\\{i}.csv",
                        encoding="utf-8-sig")
        STD.append(t.indiRating[:100].std())
        
    except:
        STD.append("NaN")
        
# 插入新的数据 STD 标准差     


In [23]:
filecsv.insert(8,column="STD",value=STD)

- 输出数据 

In [24]:
filecsv

,id,city,ctripName,matchingNum,matchingLabel,value,stars,price,STD,价格差百分比
0,345001,上海,上海日航饭店,1,1,4.4,5,970,1.16969,0.206186
1,375528,上海,上海南新雅大酒店,1,0,4.5,5,968,1.07985,0.206186
2,345008,广州,广州方圆奥克伍德豪景,2,1,4.5,0,1040,0.771133,2.980769
3,456162,广州,广州雅诗阁服务公寓,2,0,4.5,0,1071,0.836527,2.980769
4,345012,广州,桔子水晶酒店(广州淘金店),3,1,4.8,0,826,0.67577,0.242131
5,1317133,广州,广州大学城雅乐轩酒店,3,0,4.5,0,828,0.947551,0.242131
6,345025,北京,北京中奥马哥孛罗大酒店,4,1,4.8,5,926,1.02555,1.079914
7,374795,北京,北京伯豪瑞廷酒店,4,0,4.6,5,936,1.24048,1.079914
8,345029,北京,北京西单美爵酒店,5,1,4.5,4,788,0.94313,0.126904
9,1194548,北京,北京锡华商务酒店,5,0,4.2,4,789,1.34292,0.126904


In [25]:
filecsv.to_csv("[4] H1 matching with STD.csv",encoding="utf-8-sig")

In [26]:
def tocsv2(filename):
    tager=pd.read_csv(f"{filename}.csv",encoding="utf-8-sig")

    x=tager.iloc[0::2,1:]
    x.reset_index(drop="index",inplace=True)

    y=tager.iloc[1::2,1:]
    y.columns=[(i+"_matched Result") for i in y.columns]
    y.reset_index(drop="index",inplace=True)

    z=pd.concat([x,y],axis=1)
    z.to_csv(f"{filename}_marge.csv",encoding="utf-8-sig")
    
tocsv2("[4] H1 matching with STD")

## 输出包含所有评论的数据

In [31]:
SerTD = []
H1data = pd.read_csv("[4] H1 matching with STD.csv", encoding="utf-8-sig")
# 去除无效数据

H1data.head()
listname=["id","city","ctripName","matchingNum","matchingLabel","value","stars","price","价格差百分比","STD"]
H1data=H1data[listname]
H1data

,id,city,ctripName,matchingNum,matchingLabel,value,stars,price,价格差百分比,STD
0,345001,上海,上海日航饭店,1,1,4.4,5,970,0.206186,1.169691
1,375528,上海,上海南新雅大酒店,1,0,4.5,5,968,0.206186,1.079850
2,345008,广州,广州方圆奥克伍德豪景,2,1,4.5,0,1040,2.980769,0.771133
3,456162,广州,广州雅诗阁服务公寓,2,0,4.5,0,1071,2.980769,0.836527
4,345012,广州,桔子水晶酒店(广州淘金店),3,1,4.8,0,826,0.242131,0.675770
5,1317133,广州,广州大学城雅乐轩酒店,3,0,4.5,0,828,0.242131,0.947551
6,345025,北京,北京中奥马哥孛罗大酒店,4,1,4.8,5,926,1.079914,1.025550
7,374795,北京,北京伯豪瑞廷酒店,4,0,4.6,5,936,1.079914,1.240481
8,345029,北京,北京西单美爵酒店,5,1,4.5,4,788,0.126904,0.943130
9,1194548,北京,北京锡华商务酒店,5,0,4.2,4,789,0.126904,1.342924


In [32]:
g1=H1data.id.tolist()
for i in range(len(g1)):
    try:
        t = pd.read_csv(f"{os.getcwd()}\\rating2\\{g1[i]}.csv", encoding="utf-8-sig")
    except:
        a = [H1data.iloc[i:i+1, :]]
        dfe = pd.concat(a, axis=0)
        dfe.insert(5, column="用户评分", value=["NaN"])
        dfe.insert(5, column="用户序号", value=["NaN"])
        SerTD.append(dfe)


    a = [H1data.iloc[i:i+1, :] for _ in range(t.indiRating[:100].shape[0])]
    dfe = pd.concat(a, axis=0)

    dfe.insert(5, column="用户评分", value=(t.indiRating[:100]).tolist())
    dfe.insert(5, column="用户序号", value=[j+1 for j in range(t.indiRating[:100].shape[0])])
    if i%500==0:
        print(i)
    SerTD.append(dfe)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500


In [33]:
dfe=pd.concat(SerTD,axis=0)
dfe.reset_index(drop="index",inplace=True)
dfe.head()

,id,city,ctripName,matchingNum,matchingLabel,用户序号,用户评分,value,stars,price,价格差百分比,STD
0,345001,上海,上海日航饭店,1,1,1,4.5,4.4,5,970,0.206186,1.169691
1,345001,上海,上海日航饭店,1,1,2,4.3,4.4,5,970,0.206186,1.169691
2,345001,上海,上海日航饭店,1,1,3,3,4.4,5,970,0.206186,1.169691
3,345001,上海,上海日航饭店,1,1,4,5,4.4,5,970,0.206186,1.169691
4,345001,上海,上海日航饭店,1,1,5,3.8,4.4,5,970,0.206186,1.169691


- 从上往下阅读

In [34]:
dfe.to_csv("[4] H1 matching with allrating.csv",encoding="utf-8-sig")

- 左右两边阅读

In [35]:
ret=[]
for i in range(1,int(dfe.matchingNum.max())+1):
    t = dfe[dfe.matchingNum == i]
    T = t[t.ctripName == t.iloc[1].ctripName]
    T.reset_index(drop="index", inplace=True)
    TT = t[t.ctripName != t.iloc[1].ctripName]
    TT.columns=[(str(i)+"RM") for i in TT.columns.tolist()]
    TT.reset_index(drop="index", inplace=True)
    temp=pd.concat([T, TT], axis=1)
    ret.append(temp)

In [36]:
matchingwithallrating=pd.concat(ret,sort=False, axis=0)
matchingwithallrating.reset_index(drop="index", inplace=True)

In [37]:
matchingwithallrating.to_csv("[4] H1 matching with allrating_marge.csv",encoding="utf-8-sig")

---